# Import libraries

In [5]:
import time
import requests
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
import json
import pandas as pd
import csv

# Get Cache

# Concurrency

# Get Sessions

In [6]:
session = requests.Session()

# Get Urls

In [7]:
def get_page_links(url):
    driver = webdriver.Chrome()
    driver.implicitly_wait(10)
    driver.get(url)
    driver.find_element(By.XPATH,'//*[@id="uc-btn-accept-banner"]').click()

    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    s_links = soup.select('li.search-results__item a')

    url_list = []
    for element in s_links:
        link = element["href"]
        if '/agency' in link or '/en/classified/' not in link: 
            pass

        elif 'https://www.immoweb.be' not in link:
            link = 'https://www.immoweb.be' + link
            url_list.append(link)
        else:
            url_list.append(link)
    return url_list


def get_element_url_list():
    all_urls = []
    for i,x in enumerate(range(1,15)):
        urls = get_page_links(f'https://www.immoweb.be/en/search/house-and-apartment/for-sale?countries=BE&page={x}&orderBy=relevance')
        for url in urls:
            start = time.time()
            if url not in all_urls:
                all_urls.append(url)
            end = time.time()
        print('got Url',i,end-start)
    print(len(all_urls))
    return all_urls

#get houses
#get appartsments

In [8]:
url_list = get_element_url_list()

got Url 0 9.5367431640625e-07
got Url 1 7.152557373046875e-07
got Url 2 1.1920928955078125e-06
got Url 3 2.6226043701171875e-06
got Url 4 3.337860107421875e-06
got Url 5 3.5762786865234375e-06
got Url 6 4.291534423828125e-06
got Url 7 3.337860107421875e-06
got Url 8 3.0994415283203125e-06
got Url 9 8.58306884765625e-06
got Url 10 1.430511474609375e-05
got Url 11 6.9141387939453125e-06
got Url 12 9.5367431640625e-06
got Url 13 1.049041748046875e-05
424


# Get Elements_properties_raw

In [9]:
def get_property(url):
    req_element = session.get(url)
    soup = BeautifulSoup(req_element.text, 'html.parser')
    element_info = soup.select('div.classified script')
    text = element_info[0].text

    return json.loads("".join(("".join(text.split('=')[1:]).split(";")[:-1])))

In [10]:
main_dictionary = dict()
start = time.time()
for index,url in enumerate(url_list):
    start = time.time()
    main_dictionary[url] = get_property(url)
    end = time.time()
    print('scraped',index,' time ',end-start,url)
end = time.time()
print(end-start)

scraped 0  time  0.46035194396972656 https://www.immoweb.be/en/classified/apartment/for-sale/etterbeek/1040/10146186?searchId=633948fb82052
scraped 1  time  0.31189703941345215 https://www.immoweb.be/en/classified/apartment/for-sale/woluwe-saint-lambert/1200/10145269?searchId=633948fb82052
scraped 2  time  0.34310340881347656 https://www.immoweb.be/en/classified/new-real-estate-project-apartments/for-sale/gent-sint-denijs-westrem/9051/10145286?searchId=633948fb82052
scraped 3  time  0.32053422927856445 https://www.immoweb.be/en/classified/apartment/for-sale/ixelles/1050/10141214?searchId=633948fb82052
scraped 4  time  0.39783263206481934 https://www.immoweb.be/en/classified/new-real-estate-project-apartments/for-sale/brussels-city/1000/9881253?searchId=633948fb82052
scraped 5  time  0.28223633766174316 https://www.immoweb.be/en/classified/apartment/for-sale/laeken/1020/10143541?searchId=633948fb82052
scraped 6  time  0.26824092864990234 https://www.immoweb.be/en/classified/apartment/fo

In [11]:
for index,url in enumerate(url_list):
    print(index,url)

0 https://www.immoweb.be/en/classified/apartment/for-sale/etterbeek/1040/10146186?searchId=633948fb82052
1 https://www.immoweb.be/en/classified/apartment/for-sale/woluwe-saint-lambert/1200/10145269?searchId=633948fb82052
2 https://www.immoweb.be/en/classified/new-real-estate-project-apartments/for-sale/gent-sint-denijs-westrem/9051/10145286?searchId=633948fb82052
3 https://www.immoweb.be/en/classified/apartment/for-sale/ixelles/1050/10141214?searchId=633948fb82052
4 https://www.immoweb.be/en/classified/new-real-estate-project-apartments/for-sale/brussels-city/1000/9881253?searchId=633948fb82052
5 https://www.immoweb.be/en/classified/apartment/for-sale/laeken/1020/10143541?searchId=633948fb82052
6 https://www.immoweb.be/en/classified/apartment/for-sale/hoboken/2660/10143954?searchId=633948fb82052
7 https://www.immoweb.be/en/classified/new-real-estate-project-apartments/for-sale/zedelgem/8210/10074861?searchId=633948fb82052
8 https://www.immoweb.be/en/classified/new-real-estate-project-a

# Save into json

In [12]:
with open("Main_datas.json","w") as main_data_file:
    main_data_file.write(json.dumps(main_dictionary))

# Get Elements_properties_list

In [13]:
data_main_list = list()
start = time.time()
for index,element in enumerate(main_dictionary.values()):
    element_data = {
    'id': element['id'],
    'url': list(main_dictionary.keys())[index],
    'Locality': element['property']['location']['postalCode'] if element['property']['location'] is not None else None,
    'Type of property': element['property']['type'] if element['property']['type'] is not None else None,
    'Subtype of property': element['property']['subtype'] if element['property']['subtype'] is not None else None,
    'Price': element['price']['mainValue'] if element['price'] is not None else None,
    'Type of sale': element['price']['type'] if element['price'] is not None else None,
    'Number of rooms': element['property']['roomCount'] if element['property']  is not None else None,
    'Area': element['property']['netHabitableSurface'] if element['property'] is not None else None,
    'Fully equipped kitchen': element['property']['kitchen']['type'] if element['property']['kitchen'] is not None else None,
    'Furnished': element['transaction']['sale']['isFurnished'] if element['transaction']['sale'] is not None else None,
    'Open fire': element['property']['fireplaceExists'] if element['property'] is not None else None,
    'Terrace': element['property']['hasTerrace'] if element['property'] is not None else None,
    'Terrace area': element['property']['terraceSurface'] if element['property'] is not None else None,
    'Garden': element['property']['hasGarden'] if element['property'] is not None else None,
    'Garden area': element['property']['gardenSurface'] if element['property'] is not None else None,
    'Surface of the land': element['property']['land']['surface'] if element['property']['land'] is not None else None,
    'Surface area of the plot of land': element['property']['land']['surface'] if element['property']['land'] is not None else None,
    'Number of facades ': element['property']['building']['facadeCount'] if element['property']['building'] is not None else 1,
    'Swimming pool': element['property']['hasSwimmingPool'] if element['property'] is not None else None,
    'State of the building': element['property']['building']['condition'] if element['property']['building'] is not None else None
    }
    data_main_list.append(element_data)
    print('Done',list(main_dictionary.keys())[index])
    
end = time.time()
print(end-start)

Done https://www.immoweb.be/en/classified/apartment/for-sale/etterbeek/1040/10146186?searchId=633948fb82052
Done https://www.immoweb.be/en/classified/apartment/for-sale/woluwe-saint-lambert/1200/10145269?searchId=633948fb82052
Done https://www.immoweb.be/en/classified/new-real-estate-project-apartments/for-sale/gent-sint-denijs-westrem/9051/10145286?searchId=633948fb82052
Done https://www.immoweb.be/en/classified/apartment/for-sale/ixelles/1050/10141214?searchId=633948fb82052
Done https://www.immoweb.be/en/classified/new-real-estate-project-apartments/for-sale/brussels-city/1000/9881253?searchId=633948fb82052
Done https://www.immoweb.be/en/classified/apartment/for-sale/laeken/1020/10143541?searchId=633948fb82052
Done https://www.immoweb.be/en/classified/apartment/for-sale/hoboken/2660/10143954?searchId=633948fb82052
Done https://www.immoweb.be/en/classified/new-real-estate-project-apartments/for-sale/zedelgem/8210/10074861?searchId=633948fb82052
Done https://www.immoweb.be/en/classifie

In [14]:
print (data_main_list)

[{'id': 10146186, 'url': 'https://www.immoweb.be/en/classified/apartment/for-sale/etterbeek/1040/10146186?searchId=633948fb82052', 'Locality': '1040', 'Type of property': 'APARTMENT', 'Subtype of property': 'APARTMENT', 'Price': 275000, 'Type of sale': 'residential_sale', 'Number of rooms': None, 'Area': 75, 'Fully equipped kitchen': 'INSTALLED', 'Furnished': False, 'Open fire': True, 'Terrace': True, 'Terrace area': 9, 'Garden': None, 'Garden area': None, 'Surface of the land': None, 'Surface area of the plot of land': None, 'Number of facades ': 2, 'Swimming pool': None, 'State of the building': 'TO_BE_DONE_UP'}, {'id': 10145269, 'url': 'https://www.immoweb.be/en/classified/apartment/for-sale/woluwe-saint-lambert/1200/10145269?searchId=633948fb82052', 'Locality': '1200', 'Type of property': 'APARTMENT', 'Subtype of property': 'APARTMENT', 'Price': 275000, 'Type of sale': 'residential_sale', 'Number of rooms': None, 'Area': 83, 'Fully equipped kitchen': 'HYPER_EQUIPPED', 'Furnished': 

# Save information into CSV

In [17]:
def save_to_csv(list_dict):
    """a function that prints a csv starting from a dictionarires list 
        main(list_dict) """
    # we need at least 2 dict to create a csv file, so I made a df from 2 dict (the first 2 dict that I have)
    df = pd.DataFrame([list_dict[0],list_dict[1]])
    df.to_csv("df_immo.csv", index=False)
    #
    for i in  range(2, len(data_main_list)): #
        df = pd.read_csv("df_immo.csv")
        #I don't know if we are already cheacking for duplicates, but this will do that
        if data_main_list[i]["id"] not in df["id"].array:
            #open csv in append mode ("a")
            file = open('df_immo.csv', "a")
            #use csv library to add a row to the csv file. The row has to be in a list format, so I've used the .values() method of the ditionary class
            csv.writer(file).writerow(data_main_list[i].values())           
    #at the end close the csv file
    file.close

It took 2,68 seconds for 400 houses, should take around 67 seconds for 10,000

Not Checking if the file exists 0:53

Checking if the file exists 0,59

In [19]:
start = time.time()
for i in range(25):    
    save_to_csv(data_main_list)
end = time.time()
print(end-start)